<img src="./images/DLI_Header.png" width=400/>

# Fundamentals of Accelerated Data Science # 

## 02 - Preparing Data for Graph Construction ##

**Table of Contents**
<br>
This notebook introduces the basics of representing and constructing a graph. This notebook covers the below sections:
1. [Background](#Background)
2. [Environment](#Environment)
3. [Read Data](#Read-Data)
    * [UK Road Nodes](#UK-Road-Nodes)
    * [UK Road Edges](#UK-Road-Edges)
    * [Exercise #1 - Make IDs Compatible](#Exercise-#1---Make-IDs-Compatible)
5. [Data Summary](#Data-Summary)
6. [Building the Road Network Graph](#Building-the-Road-Network-Graph)
    * [Reindex `road_nodes`](#Reindex-road_nodes)
    * [Analyzing the Graph](#Analyzing-the-Graph)
7. [Construct a Graph of Roads with Time Weights](#Construct-a-Graph-of-Roads-with-Time-Weights)
    * [Road Type to Speed Conversion](#Road-Type-to-Speed-Conversion)
    * [Step 1: Merge `speed_gdf` into `road_edges`](#Step-1:-Merge-speed_gdf-into-road_edges)
    * [Exercse #2 - Step 2: Add Length in Seconds Column](#Exercse-#2---Step-2:-Add-Length-in-Seconds-Column)
    * [Exercise #3 - Step 3: Construct the Graph](#Exercise-#3---Step-3:-Construct-the-Graph)

## Background ##
As part of our larger data science goal for this workshop, we will be working with data reflecting the entire road network of Great Britain. We have as a starting point road data extracted into tabular csv format from official [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files. Ultimately, we would like to use cuGraph to perform GPU-accelerated graph analytics on this data, but in order to do so, we need to do some preprocessing to get it ready for graph creation.

In this notebook you will be learning additional cuDF data transformation techniques in a demonstration of prepping data for ingestion by cuGraph. Next, you will do a series of exercises to perform a similar transformation of the data for the creation of a graph with different edge weights.

## Environment ##

In addition to `cudf`, for this notebook we will also import `cugraph`, which we will use (after data preparation) to construct a GPU-accelerated graph. We also import `networkx` for a brief performance comparison later on.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import cudf
import cugraph as cg

import networkx as nx

## Read Data ##

In this notebook we will be working with two data sources that will help us create a graph of the UK's road networks.

### UK Road Nodes ###

The first data table describes the nodes in the road network: endpoints, junctions (including roundabouts), and points that break up a long stretch of curving road so that it can be mapped correctly (instead of as a straight line).

The coordinates for each point are in the OSGB36 format we explored earlier in section 1-05.

In [2]:
road_nodes = cudf.read_csv('./data/road_nodes.csv')
road_nodes.head()

,node_id,east,north,type
0,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,320608.0938,870994.0000,junction
1,id634D65C1-C38B-4868-9080-2E1E47F0935C,320628.5000,871103.8125,road end
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,320635.4688,870983.9375,junction
3,id51555819-1A39-4B41-B0C9-C6D2086D9921,320648.7188,871083.5625,junction
4,id9E362428-79D7-4EE3-B015-0CE3F6A78A69,320658.1875,871162.3750,junction


In [3]:
road_nodes.dtypes

node_id     object
east       float64
north      float64
type        object
dtype: object

In [4]:
road_nodes.shape

(3121148, 4)

In [5]:
road_nodes['type'].unique()

0       junction
1       road end
2    pseudo node
3     roundabout
Name: type, dtype: object

### UK Road Edges ###

The second data table describes road segments, including their start and end points, how long they are, and what kind of road they are.

In [6]:
road_edges = cudf.read_csv('./data/road_edges.csv')
road_edges.head()

,src_id,dst_id,length,type,form
0,#id138447A5-91D4-4642-BFAC-13F309705429,#id84C9DAD4-9243-4742-B582-E8CBC848E08A,314,Restricted Local Access Road,Single Carriageway
1,#idD615F9C5-5BE9-412D-9FED-F4928BAB4146,#idA1BB20B9-0751-4B42-9925-20607ABF5027,104,Restricted Local Access Road,Single Carriageway
2,#idDC14D4D1-774E-487D-8EDE-60B129E5482C,#id51555819-1A39-4B41-B0C9-C6D2086D9921,100,Restricted Local Access Road,Single Carriageway
3,#id626FC567-199C-41FB-9F29-1AB718874128,#idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B,93,Restricted Local Access Road,Single Carriageway
4,#id03312900-B147-4CA3-A858-E2BF6AD1ECA7,#id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,95,Restricted Local Access Road,Single Carriageway


In [7]:
road_edges.dtypes

src_id    object
dst_id    object
length     int64
type      object
form      object
dtype: object

In [8]:
road_edges.shape

(3725531, 5)

In [9]:
road_edges['type'].unique()

0    Restricted Local Access Road
1                      Local Road
2                          B Road
3           Secondary Access Road
4                      Minor Road
5                          A Road
6               Local Access Road
7                        Motorway
Name: type, dtype: object

In [10]:
road_edges['form'].unique()

0            Single Carriageway
1    Collapsed Dual Carriageway
2                    Roundabout
3                     Slip Road
4        Shared Use Carriageway
5              Dual Carriageway
6                 Guided Busway
Name: form, dtype: object

### Exercise #1 - Make IDs Compatible ###

Our csv files were derived from original [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) files, and as you can see from the above, both `road_edges['src_id']` and `road_edges['dst_id']` contain a leading `#` character that `road_nodes['node_id']` does not. To make the IDs compatible between the edges and nodes, use cuDF's [string method](https://docs.rapids.ai/api/nvstrings/stable/) `.str.lstrip` to replace the `src_id` and `dst_id` columns in `road_edges` with values stripped of the leading `#` characters.

In [11]:
road_edges['src_id'] = road_edges['src_id'].str.lstrip('#')
road_edges['dst_id'] = road_edges['dst_id'].str.lstrip('#')
road_edges.head()

,src_id,dst_id,length,type,form
0,id138447A5-91D4-4642-BFAC-13F309705429,id84C9DAD4-9243-4742-B582-E8CBC848E08A,314,Restricted Local Access Road,Single Carriageway
1,idD615F9C5-5BE9-412D-9FED-F4928BAB4146,idA1BB20B9-0751-4B42-9925-20607ABF5027,104,Restricted Local Access Road,Single Carriageway
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,id51555819-1A39-4B41-B0C9-C6D2086D9921,100,Restricted Local Access Road,Single Carriageway
3,id626FC567-199C-41FB-9F29-1AB718874128,idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B,93,Restricted Local Access Road,Single Carriageway
4,id03312900-B147-4CA3-A858-E2BF6AD1ECA7,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,95,Restricted Local Access Road,Single Carriageway


Click ... for solution. 

## Data Summary ##

Now that the data is cleaned we can see just how many roads and endpoints/junctions/curve points we will be working with, as well as its memory footprint in our GPU. The GPUs we are using can hold and analyze much larger graphs than this one!

In [12]:
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

3725531 edges, 3121148 nodes


In [13]:
!nvidia-smi

Mon Dec 23 12:38:57 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1B.0 Off |                    0 |
| N/A   25C    P0    24W /  70W |   2964MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

## Building the Road Network Graph ##

We don't have information on the direction of the roads (some of them are one-way), so we will assume all of them are two-way for simplicity. That makes the graph "undirected," so we will build a cuGraph `Graph` rather than a directed graph or`DiGraph`.

We initialize it with edge sources, destinations, and attributes, which for our data will be the length of the roads:

In [14]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_edges, source='src_id', destination='dst_id', edge_attr='length')

CPU times: user 439 ms, sys: 91.5 ms, total: 530 ms
Wall time: 528 ms


Just as a point of comparison, we also construct the equivalent graph in NetworkX from the equivalent cleaned and prepped Pandas dataframe.

In [15]:
road_edges_cpu = road_edges.to_pandas()
%time G_cpu = nx.convert_matrix.from_pandas_edgelist(road_edges_cpu, source='src_id', target='dst_id', edge_attr='length')

KeyboardInterrupt: 

### Reindex `road_nodes` ###

For efficient lookup later, we will reindex `road_nodes` to use the `node_id` as its index - remember, we will typically get results from the graph analytics in terms of `node_id`s, so this lets us easily pull other information about the nodes (like their locations). We then sort the dataframe on this new index:

In [16]:
road_nodes = road_nodes.set_index('node_id', drop=True)
%time road_nodes = road_nodes.sort_index()
road_nodes.head()

CPU times: user 270 ms, sys: 8.84 ms, total: 279 ms
Wall time: 279 ms


,east,north,type
node_id,,,
id000000F5-5180-4C03-B05D-B01352C54F89,432920.250,572547.4375,road end
id000003F8-9E09-4829-AD87-6DA4438D22D8,526616.375,189678.3906,junction
id000010DA-C89A-4198-847A-6E62815E038A,336879.000,731824.0000,junction
id000017A0-1843-4BC7-BCF7-C943B6780839,380635.000,390153.0000,junction
id00001B2A-155F-4CD3-8E06-7677ADC6AF74,337481.000,350509.7188,junction


### Analyzing the Graph ###

Now that we have created the graph we can analyze the number of nodes and edges in it:

In [17]:
G.number_of_nodes()

3078117

In [18]:
G.number_of_edges()

1875790

Notice that the number of edges is slightly smaller than the number of edges in `road_edges` printed above--the original data came from map tiles, and roads that passed over the edge of a tile were listed in both tiles, so cuGraph de-duplicated them. If we were creating a `MultiGraph` or `MultiDiGraph`--a graph that can have multiple edges in the same direction between nodes--then duplicates could be preserved.

We can also analyze the degrees of our graph nodes:

In [19]:
deg_df = G.degree()

In an undirected graph, every edge entering a node is simultaneously an edge leaving the node, so we expect the nodes to have a minimum degree of 2:

In [20]:
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

You will spend more time using this GPU-accelerated graph later in the workshop.

## Construct a Graph of Roads with Time Weights ##

For this series of exercises, you are going to construct and analyze a new graph of Great Britain's roads using the techniques just demonstrated, but this time, instead of using raw distance for the edges' weights, you will be using the time it will take to travel between the two nodes at a notional speed limit.

You will be beginning this exercise with the `road_edges` dataframe from earlier:

In [21]:
road_edges.dtypes

src_id    object
dst_id    object
length     int64
type      object
form      object
dtype: object

### Road Type to Speed Conversion ###

In order to calculate how long it should take to travel along a road, we need to know its speed limit. We will do this by utilizing `road_edges['type']`, along with rules for the speed limits for each type of road.

Here are the unique types of roads in our data:

In [22]:
road_edges['type'].unique()

0    Restricted Local Access Road
1                      Local Road
2                          B Road
3           Secondary Access Road
4                      Minor Road
5                          A Road
6               Local Access Road
7                        Motorway
Name: type, dtype: object

And here is a table with assumptions about speed limits we can use for our conversion:

In [23]:
# https://www.rac.co.uk/drive/advice/legal/speed-limits/
# Technically, speed limits depend on whether a road is in a built-up area and the form of carriageway,
# but we can use road type as a proxy for built-up areas.
# Values are in mph.

speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

We begin by creating `speed_gdf` to store each road type with its speed limit:

In [24]:
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys()
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

,type,limit_mph
0,Motorway,70
1,A Road,60
2,B Road,60
3,Local Road,30
4,Local Access Road,30
5,Minor Road,30
6,Restricted Local Access Road,30
7,Secondary Access Road,30


Next we add an additional column, `limit_m/s`, which for each road type will give us a measure of how fast one can travel on it in meters / second.

In [25]:
# We will have road distances in meters (m), so to get road distances in seconds (s), we need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

,type,limit_mph,limit_m/s
0,Motorway,70,31.292722
1,A Road,60,26.822333
2,B Road,60,26.822333
3,Local Road,30,13.411167
4,Local Access Road,30,13.411167
5,Minor Road,30,13.411167
6,Restricted Local Access Road,30,13.411167
7,Secondary Access Road,30,13.411167


### Step 1: Merge `speed_gdf` into `road_edges` ###

cuDF provides merging functionality just like Pandas. Since we will be using values in `road_edges` to construct our graph, we need to merge `speed_gdf` into `road_edges` (similar to a database join). You can merge on the `type` column, which both of these dataframes share.

In [26]:
%time road_edges = road_edges.merge(speed_gdf, on='type')

CPU times: user 50.4 ms, sys: 10.1 ms, total: 60.5 ms
Wall time: 59.2 ms


In [27]:
road_edges

,src_id,dst_id,length,type,form,limit_mph,limit_m/s
0,id3859B8E9-BC47-480F-A492-39E9EE061D05,id2161B6B5-7999-4256-881F-09DD3491D4A9,29,Local Road,Single Carriageway,30,13.411167
1,id2161B6B5-7999-4256-881F-09DD3491D4A9,idE3FD0E84-6A33-4EF2-9BBF-0581F9900C14,16,Local Road,Single Carriageway,30,13.411167
2,idD4991675-D2B9-4DAC-84AF-B2E5F397C03D,idBBC368A8-35CF-4B8D-8713-BAF193C9E2C8,71,Local Road,Single Carriageway,30,13.411167
3,idBBC368A8-35CF-4B8D-8713-BAF193C9E2C8,idE96E69B3-927E-480E-9A11-0130250154BA,208,Local Road,Single Carriageway,30,13.411167
4,id4C2154FE-BFDF-434E-BE70-ADB1241AFC84,id8ED13698-D1B4-4AEE-BDD2-5FC6DD0E0325,18,Local Road,Single Carriageway,30,13.411167
...,...,...,...,...,...,...,...
3725526,id79127F0E-8B33-461B-94B0-AA6483C15AC6,id0E5D1367-668E-463A-B443-CEE022395D1F,5,Local Road,Single Carriageway,30,13.411167
3725527,id0E5D1367-668E-463A-B443-CEE022395D1F,idA81A46E9-596B-4DBB-9792-4290CD21FA77,93,Local Road,Single Carriageway,30,13.411167
3725528,id79127F0E-8B33-461B-94B0-AA6483C15AC6,id0C380CAA-1CAF-4ECC-B4B3-1459D2A2E5F4,79,Restricted Local Access Road,Single Carriageway,30,13.411167
3725529,id2D4F54F7-0C31-4966-9D72-88E11F0438F2,id79127F0E-8B33-461B-94B0-AA6483C15AC6,124,Local Road,Single Carriageway,30,13.411167


### Exercse #2 - Step 2: Add Length in Seconds Column ###

You now need to calculate the number of seconds it will take to traverse a given road at the speed limit. This can be done by dividing a road's length in m by its speed limit in m/s. Perform this calculation on `road_edges` and store the results in a new column `length_s`.

In [28]:
road_edges['length_s'] = road_edges['length'] / road_edges['limit_m/s']

Click ... for solution. 

### Exercise #3 - Step 3: Construct the Graph ###

Construct a cuGraph `Graph` called `G_ex` using the sources and destinations found in `road_edges`, along with length-in-seconds values for the edges' weights.

In [30]:
G_ex = cg.Graph()
G_ex.from_cudf_edgelist(road_edges, source='src_id', destination='dst_id', edge_attr='length_s')

Click ... for solution. 

In [31]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Well Done!** Let's move to the [next notebook](3-03_cugraph.ipynb). 

<img src="./images/DLI_Header.png" width=400/>